# Transpose of a square matrix

## Finding transpose of a square matrix without using temporary matrix


In [1]:
using Pkg
Pkg.add("CUDAnative")
Pkg.add("CuArrays")
Pkg.add("CUDAdrv")
Pkg.add("BenchmarkTools")
Pkg.add("Test")

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [0]:
using CUDAnative,CuArrays,CUDAdrv,BenchmarkTools,Test

In [10]:
# Kernel to find transpose of a square matrix without using a temporary matrix

function transpose(d_input,N)
  idx = (blockIdx().x - 1) * blockDim().x + threadIdx().x
  idy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
  
  # Bounds check 
  if(idx <= N && idy <= N && idx <= idy)
    temp = d_input[idx,idy]
    d_input[idx,idy] = d_input[idy,idx]
    d_input[idy,idx] = temp
  end
  
  return 
end

transpose (generic function with 1 method)

In [20]:
N = 100
input = rand(N,N)
d_input=CuArray(input)

100×100 CuArray{Float64,2}:
 0.89954   0.826267    0.850579   …  0.0920916   0.256064   0.0727674
 0.501696  0.238456    0.720401      0.583565    0.390627   0.777108 
 0.167341  0.106352    0.0643141     0.172838    0.0535584  0.83691  
 0.7532    0.643455    0.102143      0.224618    0.370367   0.0678088
 0.367258  0.388475    0.0599876     0.917886    0.735512   0.32929  
 0.674111  0.701897    0.853748   …  0.574248    0.278316   0.660032 
 0.563485  0.0219543   0.768478      0.487299    0.96116    0.38714  
 0.690795  0.422564    0.443492      0.872972    0.318923   0.309138 
 0.726602  0.359488    0.0594943     0.901943    0.117709   0.531734 
 0.484494  0.685469    0.243665      0.356876    0.702318   0.7574   
 0.477603  0.979003    0.0852392  …  0.839141    0.915725   0.902608 
 0.32882   0.757064    0.751324      0.861519    0.238029   0.495303 
 0.340889  0.26378     0.852717      0.219082    0.225412   0.319702 
 ⋮                                ⋱                           

In [0]:
@cuda blocks = (Int(ceil(N/32)),Int(ceil(N/32))) threads = (32,32) transpose(d_input,N)

In [22]:
h_transpose = Array(d_input)

100×100 Array{Float64,2}:
 0.89954    0.501696   0.167341   …  0.445026   0.238674   0.288225 
 0.826267   0.238456   0.106352      0.731318   0.606609   0.170745 
 0.850579   0.720401   0.0643141     0.892249   0.668119   0.389698 
 0.963517   0.269649   0.966543      0.955812   0.758791   0.153148 
 0.914383   0.322874   0.0288655     0.0957935  0.0615485  0.910167 
 0.710737   0.164619   0.905933   …  0.0783295  0.728924   0.0336899
 0.303035   0.989227   0.238421      0.391319   0.295386   0.0308027
 0.465971   0.731124   0.838485      0.0579519  0.0943504  0.500582 
 0.815473   0.486341   0.738487      0.209123   0.591715   0.474267 
 0.331847   0.746804   0.591898      0.0853216  0.690571   0.437122 
 0.407394   0.58238    0.793981   …  0.965301   0.0410526  0.219273 
 0.0869291  0.860252   0.588181      0.579509   0.701476   0.303427 
 0.704117   0.798396   0.048976      0.762153   0.301744   0.936781 
 ⋮                                ⋱                                 
 0.72805